# Importing Modules and URL for Web Scraping

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [2]:
data

<Response [200]>

In [3]:
#data.content It will show the raw html format.

For Converting Raw html format to formatted html we will use Soup

In [4]:
soup = BeautifulSoup(data.content,"html.parser")

# Create a function to fetch the headlines from URL

In [5]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])     
    return articles

In [6]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)

Created urllist funtion to fetch all articles on different pages of website.

In [7]:
urllist

['https://www.opindia.com/latest-news/page/2/',
 'https://www.opindia.com/latest-news/page/3/',
 'https://www.opindia.com/latest-news/page/4/',
 'https://www.opindia.com/latest-news/page/5/',
 'https://www.opindia.com/latest-news/page/6/',
 'https://www.opindia.com/latest-news/page/7/',
 'https://www.opindia.com/latest-news/page/8/',
 'https://www.opindia.com/latest-news/page/9/',
 'https://www.opindia.com/latest-news/page/10/',
 'https://www.opindia.com/latest-news/page/11/',
 'https://www.opindia.com/latest-news/page/12/',
 'https://www.opindia.com/latest-news/page/13/',
 'https://www.opindia.com/latest-news/page/14/',
 'https://www.opindia.com/latest-news/page/15/',
 'https://www.opindia.com/latest-news/page/16/',
 'https://www.opindia.com/latest-news/page/17/',
 'https://www.opindia.com/latest-news/page/18/',
 'https://www.opindia.com/latest-news/page/19/',
 'https://www.opindia.com/latest-news/page/20/']

In [8]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

# Preprocessing of the Data

In [9]:
p_art =[]
for i in all_articles:
    q = i.upper()
    import re
    q = re.sub("[^A-Z0-9 ]","",q)
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

# Vectorize the Data

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

# Clustering of the Data

In [11]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [12]:
cl_res.labels_

array([3, 2, 0, ..., 1, 1, 1])

In [13]:
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [15]:
Q

,Article,Cluster
0,BJP PASMANDA OUTREACH COULD BE A GAME CHANGER...,3
1,THE ANATOMI OF PROTEST THE DEVIL THE PEOPL TH...,2
2,PASMANDA OUTREACH BY BJP OBC MORCHA DEMAND 50...,0
3,AAP MLA AMANATULLAH KHAN COME TO TERRORACCUS ...,1
4,NIRBHAYA MOTHER SLAM ASHOK GEHLOT WHO CONTINU...,1
...,...,...
1971,EKNATH SHIND GOVT IN MAHARASHTRA WIN THE FLO...,1
1972,AMRAVATI BEFOR KOLH MURDER THREE MORE PEOPL H...,1
1973,DMK MP A RAJA EMBOLDEN RAHUL GANDHI CONCEPT O...,1
1974,HINDU LIVE MATTER IS A DANGER SLOGAN AFTER KA...,1


In [16]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [17]:
R = []
for i in Q.Cluster:
    R.append(E[i])

Q['category'] = R

In [18]:
Q

,Article,Cluster,category
0,BJP PASMANDA OUTREACH COULD BE A GAME CHANGER...,3,sports
1,THE ANATOMI OF PROTEST THE DEVIL THE PEOPL TH...,2,religion
2,PASMANDA OUTREACH BY BJP OBC MORCHA DEMAND 50...,0,geopolitics
3,AAP MLA AMANATULLAH KHAN COME TO TERRORACCUS ...,1,politics
4,NIRBHAYA MOTHER SLAM ASHOK GEHLOT WHO CONTINU...,1,politics
...,...,...,...
1971,EKNATH SHIND GOVT IN MAHARASHTRA WIN THE FLO...,1,politics
1972,AMRAVATI BEFOR KOLH MURDER THREE MORE PEOPL H...,1,politics
1973,DMK MP A RAJA EMBOLDEN RAHUL GANDHI CONCEPT O...,1,politics
1974,HINDU LIVE MATTER IS A DANGER SLOGAN AFTER KA...,1,politics
